Kunstmuseen in Hessen

SQLite Datenbank zu Kunstmuseen in Hessen implementieren

Query: Wikidata:SPARQL query service
Ergebnisse befinden sich in: Kunstmuseum.json
Output: Kunstmuseen_Hessen.db

Query:

SELECT ?item ?name ?description ?image ?url (MIN(?lat) AS ?latitude) (MIN(?lon) AS ?longitude) (GROUP_CONCAT(DISTINCT ?directorLabel; SEPARATOR=", ") AS ?directors) ?foundingYear
WHERE {
  hint:Query hint:optimizer "None" .
  ?item wdt:P131* wd:Q1199 .                     # Hessen
  ?item wdt:P31/wdt:P279* wd:Q33506 .           # Museen
  ?item wdt:P31/wdt:P279* wd:Q207694 .          # Kunstmuseen
  ?item p:P625 ?coordinate .
  OPTIONAL { ?item wdt:P18 ?image . }
  OPTIONAL { ?item wdt:P856 ?url . }

  OPTIONAL {
    ?item schema:description ?descDe .
    FILTER(LANG(?descDe) = "de")
  }

  OPTIONAL {
    ?item schema:description ?descEn .
    FILTER(LANG(?descEn) = "en")
  }

  BIND(COALESCE(?descDe, ?descEn) AS ?description)

  ?coordinate psv:P625 ?coordinate_node .
  ?coordinate_node wikibase:geoLatitude ?lat .
  ?coordinate_node wikibase:geoLongitude ?lon .

  OPTIONAL {
    ?item wdt:P1037 ?directorItem .
    ?directorItem rdfs:label ?directorLabel .
    FILTER(LANG(?directorLabel) = "de")
  }

  OPTIONAL {
    ?item wdt:P571 ?foundingYear .
  }


  SERVICE wikibase:label {
    bd:serviceParam wikibase:language "[AUTO_LANGUAGE],en" .
    ?item rdfs:label ?name .
  }

  FILTER(!isBlank(?item) && !isBlank(?name) && !isBlank(?description) && !isBlank(?image) && !isBlank(?url) && !isBlank(?lat) && !isBlank(?lon))
}
GROUP BY ?item ?name ?description ?image ?url ?foundingYear
ORDER BY ASC(?name)

Libraries für sqlite3 und JSON importieren,
Ergebnisse in Datenbank abspeichern:

In [1]:
import sqlite3
import json

filename = "Kunstmuseen_Hessen.db"
db = sqlite3.connect(filename, detect_types=sqlite3.PARSE_DECLTYPES | sqlite3.PARSE_COLNAMES)

c = db.cursor()

# Create table 'Kunstmuseum'
c.execute('DROP TABLE IF EXISTS Kunstmuseum')
c.execute('''
          CREATE TABLE Kunstmuseum (
              Item varchar PRIMARY KEY,
              name varchar,
              description varchar,
              image varchar,
              article varchar,
              latitude REAL,
              longitude REAL,
              directors varchar,
              foundingYear REAL
         )
''')



print(c)

with open('Kunstmuseum.json', 'r', encoding="utf-8") as f:
    json_content = f.read()
    data = json.loads(json_content)
    

    results = data['results']['bindings']  # Access the 'bindings' key within 'results' to get the data entries

    for entry in results:
        t = (
            entry['item']['value'][32:],
            entry['name']['value'],
            entry['description']['value'],
            entry.get('image', {}).get('value'),
            entry.get('article', {}).get('value'),
            entry.get('latitude', {}).get('value'),
            entry.get('longitude', {}).get('value'),
            entry.get('directors', {}).get('value'),
            entry.get('foundingYear', {}).get('value')
        )
        # Delete the existing row with the same item value
        c.execute('DELETE FROM Kunstmuseum WHERE item = ?', (t[0],))

        # Insert the new row
        c.execute('INSERT INTO Kunstmuseum (item, name, description, image, article,latitude, longitude, directors, foundingYear) VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?)', t)

        db.commit()

results = c.execute('SELECT * FROM Kunstmuseum')
for row in results:
    print(row)

c.close()





('636924', 'Archäologisches Museum Frankfurt', 'Museum der Archäologie in Hessen', 'http://commons.wikimedia.org/wiki/Special:FilePath/Archaeologisches%20Museum%20Frankfurt%20Neubau.jpg', 'https://www.archaeologisches-museum-frankfurt.de/', 50.1089, 8.67853, 'Egon Wamers', '1937-06-22T00:00:00Z')
('1525564', 'Caricatura Museum für Komische Kunst', 'Museum für die Komische Kunst der Neuen Frankfurter Schule und weiterer zeitgenössischer Satiriker und Karikaturisten in Frankfurt am Main', 'http://commons.wikimedia.org/wiki/Special:FilePath/Eingang%20Caricatura%20Leinwandhaus.jpg', 'http://www.caricatura-museum.de/', 50.1102, 8.68511, '', None)
('885179', 'Deutsches Architekturmuseum', 'Museum in Frankfurt am Main', 'http://commons.wikimedia.org/wiki/Special:FilePath/Deutsches%20Architekturmuseum%2C%20Frankfurt.jpg', 'https://www.dam-online.de/', 50.104975, 8.6773143, 'Peter Cachola Schmal', None)
('1205606', 'Deutsches Elfenbeinmuseum', 'Museum in Erbach im Odenwald', 'http://commons.wik